In [1]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

DEBUG:__main__:hello


In [2]:
# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

DEBUG:lib.lab_lib:hello


# 2022年3月30日～

## 積モデルの実装
TODO
* モデル式 $ T = aXYZ + b $ の実装（Model_InfiniteProductOfProblemSizeMultipliedByProcesses_ForMultipleRegression）
    * ✅テストの実装
    * ✅本実装
* モデル式 $ T = a \frac{YZ}{X} + b $ の実装（Model_InfiniteProductOfProblemSizeDividedByProcesses_ForMultipleRegression）
    * ✅テストの実装
    * ✅本実装
* 上記のモデルにおいてMAPE表を作成する
    * Modelsクラスに追加実装
        * ✅モデル式 $ T = aXYZ + b $
        * ✅モデル式 $ T = a \frac{YZ}{X} + b $
    * 表への追加
        * ✅モデル式 $ T = aXYZ + b $
        * ✅モデル式 $ T = a \frac{YZ}{X} + b $

（T,X,Y,Zはそれぞれ、関数コール回数,プロセス数,問題サイズ1,問題サイズ2）

In [3]:
benchmark_name: str = "cg"

classes: list[str] = ["A", "B", "C", "D", "E", "F"]
processes: list[int] = [2, 4, 8, 16, 32, 64, 128, 256]

dict_column_names_JP: dict[str, str] = {
    "functionName": "関数名",
    "modelLin": "線形",
    "modelIp": "反比例",
    "modelLog": "対数",
    "modelLinAndIp": "線形＆反比例",
    "modelLinAndLog": "線形＆対数",
    "modelIpAndLin": "反比例＆線形",
    "modelIpAndLog": "反比例＆対数",
    "modelLogAndLin": "対数＆線形",
    "modelLogAndIp": "対数＆反比例",
    "modelProcessDividedByProblemSize": "プロセス数を問題サイズで割る",
    "modelProblemSizeDividedByProcess": "問題サイズをプロセス数で割る",
    "modelInfiniteProductOfProblemSizeMultipliedByProcesses": "問題サイズの変数の総乗とプロセス数とをかける",
    "modelInfiniteProductOfProblemSizeDividedByProcesses": "問題サイズの変数の総乗をプロセス数で割る",
    # "modelBasicTree": "回帰木",
}

model_names_list_JP: list[str] = list(dict_column_names_JP.values())
model_names_list_JP.remove("関数名")

model_names_list: list[str] = list(dict_column_names_JP.keys())
model_names_list.remove("functionName")

date: str = "2022年3月23日"

# コア数と問題サイズの両方を振る

rawDF: pd.DataFrame = return_rawDF_with_init_param(
    benchmark_name=benchmark_name,
    classes=classes,
    processes=processes,
    csv_dir_path="./csv_files/",
)

exp_var: list[str] = rawDF.columns.tolist()
for element_be_removed in [
    "functionName",
    "functionCallNum",
    "intBenchmarkClass",
    "benchmarkName",
    "benchmarkClass",
]:
    exp_var.remove(element_be_removed)
res_var: list[str] = ["functionCallNum"]

print(f"exp_var = {exp_var}")

returned_MAPE_table: pd.DataFrame = return_MAPE_Table_DF_from_rawDF(
    rawDF=rawDF,
    exp_var_list=exp_var,
    res_var_list=res_var,
    model_name_list=model_names_list,
)
df: pd.DataFrame = returned_MAPE_table.rename(columns=dict_column_names_JP)
df = df.set_index("関数名")
df = df.sort_index(axis=0)
df = df.sort_index(axis=1)

df = addLowestMAPEsModelNameColumn(df, version=2, model_name_list=model_names_list_JP)

print(
    df.style.set_precision(2).to_latex(
        caption=f"ベンチマークプログラム{benchmark_name.upper()}における各モデルでのMAPE",
        label=f"{date}allModelsMAPEin{benchmark_name.upper()}",
    )
)

df.style.format(precision=2).highlight_min(axis=1, subset=model_names_list_JP)

exp_var = ['process', 'na', 'nonzer', 'niter', 'shift']


/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estima

\begin{table}
\caption{ベンチマークプログラムCGにおける各モデルでのMAPE}
\label{2022年3月23日allModelsMAPEinCG}
\begin{tabular}{lrrrrrrrrrrrrrl}
 & プロセス数を問題サイズで割る & 反比例 & 反比例＆対数 & 反比例＆線形 & 問題サイズの変数の総乗とプロセス数とをかける & 問題サイズの変数の総乗をプロセス数で割る & 問題サイズをプロセス数で割る & 対数 & 対数＆反比例 & 対数＆線形 & 線形 & 線形＆反比例 & 線形＆対数 & 最適モデル \\
関数名 &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
.TAU_application & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 線形 \\
ALLOC_SPACE & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 線形 \\
CG & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 線形 \\
CONJ_GRAD & 37.55 & 13.88 & 13.73 & 14.50 & 37.69 & 38.04 & 42.76 & 14.80 & 14.72 & 15.25 & 13.16 & 12.41 & 12.76 & 線形＆反比例 \\
ICNVRT & 159.61 & 662.48 & 2504.27 & 894.57 & 93.04 & 89.90 & 119.98 & 2443.70 & 3811.49 & 815.79 & 502.12 & 6621.01 & 2283.06 & 問題サイズの変数の総乗をプロセス数で割る \\
INITIALIZE_MPI & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0

/tmp/ipykernel_17560/4026089130.py:68: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  df.style.set_precision(2).to_latex(


,プロセス数を問題サイズで割る,反比例,反比例＆対数,反比例＆線形,問題サイズの変数の総乗とプロセス数とをかける,問題サイズの変数の総乗をプロセス数で割る,問題サイズをプロセス数で割る,対数,対数＆反比例,対数＆線形,線形,線形＆反比例,線形＆対数,最適モデル
関数名,,,,,,,,,,,,,,
.TAU_application,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,線形
ALLOC_SPACE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,線形
CG,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,線形
CONJ_GRAD,37.55,13.88,13.73,14.50,37.69,38.04,42.76,14.80,14.72,15.25,13.16,12.41,12.76,線形＆反比例
ICNVRT,159.61,662.48,2504.27,894.57,93.04,89.90,119.98,2443.70,3811.49,815.79,502.12,6621.01,2283.06,問題サイズの変数の総乗をプロセス数で割る
INITIALIZE_MPI,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,線形
MAKEA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,線形
MPI_Barrier(),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,線形
MPI_Bcast(),0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,線形
